In [ ]:
# Headers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sbn

In [ ]:
# Imports the dataset from a csv file
dataset = pd.read_csv("train.csv")

In [ ]:
# Plots some of the dataset's information
dataset.describe()

In [ ]:
# Drops unwanted columns
dataset = dataset.drop(columns=['tr_attempts'])
dataset = dataset.drop(columns=['tr_id'])

In [ ]:
# Removes observations with empty data
dataset.dropna()

In [ ]:
# Balances the dataset by cutting the larger class
def cut_dataset(dataset, target_name):
	class_0 = dataset[dataset[target_name] == 0]
	class_1 = dataset[dataset[target_name] == 1]
	
	if len(class_0) > len(class_1):
		class_0 = class_0.sample(len(class_1))
	else:
		class_1 = class_1.sample(len(class_0))
	dataset = pd.concat([class_0, class_1])

	return dataset

dataset = cut_dataset(dataset, 'route_changed')
dataset.describe()

,tr_src,tr_dst,total_probes_sent,total_replies_last_hop,route_changed,date_index,seconds_since_start
count,739140.000000,739140.000000,739140.000000,739140.000000,739140.0,739140.000000,7.391400e+05
mean,52.574907,1.492118,1.600805,1.631813,0.5,54.595871,4.673526e+06
std,26.989622,1.655477,2.501717,3.113117,0.5,29.787219,2.574044e+06
min,0.000000,0.000000,1.000000,1.000000,0.0,1.000000,4.000000e+00
25%,30.000000,1.000000,1.000000,1.000000,0.0,37.000000,3.146948e+06
50%,53.000000,1.000000,1.000000,1.000000,0.5,55.000000,4.713844e+06
75%,77.000000,1.000000,1.000000,1.000000,1.0,79.000000,6.791585e+06
max,119.000000,12.000000,50.000000,77.000000,1.0,107.000000,9.165453e+06


In [ ]:
# Converts a String to a DataFrame
def str_to_dataframe(stringzinha):
	return pd.DataFrame(map(float, stringzinha[1:-1].split(',')))

# Converts a String to a list and applies a method to the list
def str_to_list_mean(stringzinha):
	return str_to_dataframe(stringzinha).mean()

def str_to_list_std(stringzinha):
	return str_to_dataframe(stringzinha).std()

dataset["mean_rtt"] = dataset["all_rtts"].apply(str_to_list_mean)
dataset["std_rtt"] = dataset["all_rtts"].apply(str_to_list_std)

AttributeError: module 'pandas' has no attribute 'mean'

In [ ]:
cor_pearson = dataset.corr(method='pearson')
plt.figure(figsize=(13, 13))
sbn.heatmap(cor_pearson, annot=True, cmap='coolwarm')
plt.show()